# 11.2 Whisper Transcript

지난시간에는 pydub을 이용해서 오디오파일을 최대 10분길이로 자르는 방법을 배웠다.

이번시간에는 이 자른 파일을 openAI API로 보내서 Whisper모델을 사용하여 녹취록을 받을 것이다.

In [2]:
import subprocess
from pydub import AudioSegment
import math

def extract_audio_from_video(video_path, audio_path):
    command = ["ffmpeg", "-i", video_path, "-vn", audio_path,]
    # 이렇게 배열로 형태로 전달하는 것은 보안때문이다. 또한 특수 문자가 포함되어 있을때도 더 편리하기 때문에
    # 배열방식이 권장된다.
    subprocess.run(command)

# extract_audio_from_video("../files/Joe_Dispenza_podcast.mp4", "../files/audio.mp3")

def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size * 60 * 1000 # 5분을 밀리초 단위로 변환
    chunks = math.ceil(len(track) / chunk_len) # ceil은 올림 할때 쓰는 함수

    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i+1) * chunk_len

        if i < chunks-1:
            splited_track = track[start_time:end_time]
            splited_track.export(f"{chunks_folder}/chunk_{i}.mp3", format="mp3")
        else:
            splited_track = track[start_time:]
            splited_track.export(f"{chunks_folder}/chunk_{i}.mp3", format="mp3")


In [3]:
cut_audio_in_chunks("../files/audio.mp3", 10, "../files/chunks")

In [ ]:
import openai

client = openai.OpenAI()

transcript = client.audio.transcriptions.create(model="whisper-1", file=open("../files/chunks/chunk_0.mp3", "rb"))
# 이 코드는 whisper AI로 오디오를 보내서 그 스크립트를 받기위해 작성한 것이다.
# 여기서 rb는 이진법으로 읽는다는 뜻이다.
print(transcript)

Transcription(text="We cannot attract anything in our life that we feel separate from. The moment you feel the lack and the separation, you're holding yourself hostage to the past. Best-selling author. He's lectured and given advanced workshops in more than 30 countries. Dr. Joe Dispenza. How I want to be. This is about conditioning my body emotionally into the future. As long as I'm feeling this emotion, I'm going to believe in that future. What is blocking us from manifesting love? We're thrilled to announce that we've reached 3 million subscribers. We're incredibly grateful for each and every one of you. If you enjoyed this episode, don't forget to hit that subscribe button so you never miss out on any of our new releases. We're dedicated to bringing you the content you love. Our team carefully analyzes what resonates most with you to bring on board the best experts and storytellers to help you improve your life. Some of your favorite topics are sleep science, weight loss, physical 

이렇게 하면 우리가 분할한 오디오 파일을 AI를 이용해 그 스크립트를 받아올 수 있게 된다.

이제는 각 파일들을 모두 스크립트로 바꿔주는 작업을 할것이다. 이를 위해 우리는 glob을 import해와야 한다.

glob이 어떻게 동작하는지는 아래 코드를 보며 익혀보자.

In [11]:
import glob

def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    return files

print(transcribe_chunks("../files/chunks", "..."))

['../files/chunks/chunk_0.mp3', '../files/chunks/chunk_1.mp3']


glob은 패턴에 맞는 모든 파일들을 읽어와서 우리가 원하는 처리를 거쳐 list형태로 반환하는 역할을 한다. 또한 전역변수를 만들어 다음번에 우리가 이 프로젝트로 작업할때 중복으로 실행이 안되게 해준다(?) -> 솔직히 잘 모르겠음...

위 코드를 보면 `../files/chunks`라는 폴더 안에서 mp3확장자를 가진 모든 파일(*로 모든파일이라고 명시함)을 찾아서 그에대해 우리가 원하는 처리를 해서 list형태로 반환하는 것을 볼 수 있다.

자 이제 우리는 파일들의 리스트를 얻었다. 이제 다시 함수마저 정의해보자.

In [14]:
def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    final_transcribe = ""
    for file in files:
        with open(file, "rb") as audio_file:
            transcript = client.audio.transcriptions.create(
                model="whisper-1", 
                file=audio_file,
            )
            final_transcribe += transcript.text
    with open(destination, "w") as file:
        file.write(final_transcribe)

transcribe_chunks("../files/chunks", "../files/transcript.txt")